In [1]:
# ! pip install -U scikit-learn scipy matplotlib

In [2]:
import numpy as np
import pandas as pd
import pickle
from collections import defaultdict
import json
from PIL import Image
import torch.nn as nn
import torch
import torchvision.transforms as transforms
import torchvision
import easydict
import config
import random
import re
import torch.utils.data as data
from sklearn.preprocessing import LabelEncoder

/home/arunava/miniconda3/envs/vqamedenv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [4]:
category_names = {'C1':'Modality', 'C2':'Plane','C3':'Organ','C4':'Abnormality','All': 'All dataset'}

In [5]:
with open(config.path+'/VQA_Med/answer_classes.json', 'r') as j:
        answer_classes = json.load(j)


l = len(answer_classes) 

print(l)

178


In [6]:
opt = easydict.EasyDict({
        "SEED":97,
        "BATCH_SIZE": 64,
        "VAL_BATCH_SIZE": 64,
        # "NUM_OUTPUT_UNITS": l,
        "MAX_QUESTION_LEN": 17,
        "IMAGE_CHANNEL": 1472,
        "INIT_LERARNING_RATE": 1e-4,
        "LAMNDA":0.0001,
        "MFB_FACTOR_NUM":5,
        "MFB_OUT_DIM":1000,
        "BERT_UNIT_NUM":768,
        "BERT_DROPOUT_RATIO":0.3,
        "MFB_DROPOUT_RATIO":0.1,
        "NUM_IMG_GLIMPSE":2,
        "NUM_QUESTION_GLIMPSE":2,
        "IMG_FEAT_SIZE":1,
        "IMG_INPUT_SIZE":224,
        "NUM_EPOCHS":10,
    })

In [7]:
path_change = config.path
seed_value = opt.SEED
print(seed_value) # 97
np.random.seed(seed_value) # return None
random.seed(seed_value) # return None
torch.manual_seed(seed_value) # return <torch._C.Generator object at 0x7f71cdf7a3d0>
torch.cuda.manual_seed(seed_value) # return None
torch.cuda.manual_seed_all(seed_value) # return None
torch.backends.cudnn.enabled = False 
''' backends.cudnn.enabled enables cudnn for some operations such as conv layers and RNNs, which can yield 
a significant speedup. The cudnn RNN implementation doesn’t support the backward operation during eval() 
and thus raises the error. You could disable cudnn for your workload (as already done) or try to call .train()
on the RNN module separately after using model.eval().'''
torch.backends.cudnn.benchmark = False
'''If your model does not change and your input sizes remain the same - then you may benefit from setting 
torch.backends.cudnn.benchmark = True.However, if your model changes: for instance, if you have layers that
are only "activated" when certain conditions are met, or you have layers inside a loop that can be iterated a 
different number of times, then setting torch.backends.cudnn.benchmark = True might stall your execution.'''
torch.backends.cudnn.deterministic = False
'''torch.backends.cudnn.deterministic=True only applies to CUDA convolution operations, and nothing else. 
Therefore, no, it will not guarantee that your training process is deterministic, since you're also using
torch.nn.MaxPool3d, whose backward function is nondeterministic for CUDA.'''


97


"torch.backends.cudnn.deterministic=True only applies to CUDA convolution operations, and nothing else. \nTherefore, no, it will not guarantee that your training process is deterministic, since you're also using\ntorch.nn.MaxPool3d, whose backward function is nondeterministic for CUDA."

In [8]:
train_path = path_change+'/VQA_Med_2019_Dataset/Train/ImageClef-2019-VQA-Med-Training/' #QAPairsByCategory/'
train_images_path = path_change+'/VQA_Med_2019_Dataset/Train/ImageClef-2019-VQA-Med-Training/Train_images/'

valid_path = path_change+'/VQA_Med_2019_Dataset/Valid/ImageClef-2019-VQA-Med-Validation/'#QAPairsByCategory/'
valid_images_path = path_change+'/VQA_Med_2019_Dataset/Valid/ImageClef-2019-VQA-Med-Validation/Val_images/'

test_path = path_change+'/VQA_Med_2019_Dataset/Test/VQAMed2019Test/' 
test_images_path = path_change+'/VQA_Med_2019_Dataset/Test/VQAMed2019Test/VQAMed2019_Test_Images/'

In [9]:
size = opt.IMG_INPUT_SIZE
size

224

In [10]:
transform = {
    phase: transforms.Compose([transforms.RandomResizedCrop(size),
                            transforms.RandomHorizontalFlip(),
                            transforms.ToTensor(),
                            transforms.Normalize((0.485,0.456,0.406),
                                                    (0.229,0.224,0.225))
    
    ])
    for phase in ['train','valid']
}

# transforms.RandomResizedCrop(224):
'''  This will extract a patch of size (224, 224) from your input image randomly.
So, it might pick this path from topleft, bottomright or anywhere in between. So, you are doing data augmentation 
in this part. Also, changing this value won't play nice with the fully-connected layers in your model,
so not advised to change this. '''

# transforms.RandomHorizontalFlip():
''' Once we have our image of size (224, 224), we can choose to flip it. 
This is another part of data augmentation. '''

# transforms.ToTensor():
''' This just converts your input image to PyTorch tensor. '''

# transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]):
'''  This is just input data scaling and these values
(mean and std) must have been precomputed for your dataset. Changing these values is also not advised. '''


test_transform = transforms.Compose([transforms.RandomResizedCrop(size),
                                        transforms.ToTensor(),
                                        transforms.Normalize((0.485,0.456,0.406),
                                                            (0.229,0.224,0.225))])
transform
test_transform

Compose(
    RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear), antialias=None)
    ToTensor()
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
)

In [11]:
# import config

In [12]:
# print(config.parse_opt())

In [13]:
import re
from typing import Set, List, Dict
from collections import defaultdict
from spacy.lang.en import English
nlp = English()

In [14]:
class Tokenizer:
    def __init__(self, vocab: Set[str], max_len: int) -> None:
        self.pad = '<pad>'
        self.bos = '<bos>'
        self.eos = '<eos>'
        self.unk = '<unk>'
        self.special_tokens = [self.pad, self.bos, self.eos, self.unk]
        self.vocab = self.special_tokens + sorted(list(vocab))
        self.vocab = {w: i for i, w in enumerate(self.vocab)}
        self.vocab = defaultdict(lambda: self.vocab[self.unk], self.vocab)
        self.idx_to_word = {v: k for k, v in self.vocab.items()}
        self.max_len = max_len
        self.vocab_dim = len(self.vocab)

    def encode(self, sent: str) -> Dict[str, list]:
        sent = sent.lower()
        tokens = [
            self.vocab[str(word)] for word in nlp.tokenizer(sent)
        ]
        tokens = tokens[:self.max_len - 2]
        # Add sentence begining and end
        tokens = [self.vocab[self.bos]] + tokens + [self.vocab[self.eos]]
        total_tokens = len(tokens)
        if total_tokens != self.max_len:
            rem = self.max_len - total_tokens
            tokens += [self.vocab[self.pad]]*rem
        return {
            'input_ids': tokens
        }

    def decode(self, tokens: List[int]) -> str:
        tokens = [self.idx_to_word[t] for t in tokens]
        decoded = [
            token
            for token in tokens
            if (token != self.vocab.pad) and
               (token != self.vocab.bos) and
               (token != self.vocab.eos)
        ]
        return ' '.join(decoded)

    @classmethod
    def from_list(cls, unique_ques_list: List[str], max_len: int):
        vocab = set([
            str(word) for sent in unique_ques_list
            for word in nlp.tokenizer(sent.lower())
        ])
        return cls(vocab, max_len=max_len)

In [15]:
# from transformers import BertTokenizer

In [16]:

# class BERTokenizer():

#     def __init__(self,opt):
#         # Load the BERT tokenizer
#         print('opt',opt)
#         self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#         self.opt = opt
    

#     #pre-process the text data
#     def text_preprocessing(self, text):

#         # Remove trailing whitespace
#         text = re.sub(r'\s+', ' ', text).strip()

#         return text


#     # Create a function to tokenize a set of texts
#     def preprocessing_for_bert(self, data):
#         # print('hm chal rhe hai')
#         """Perform required preprocessing steps for pretrained BERT.
#         @param    data (np.array): Array of texts to be processed.
#         @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
#         @return   attention_masks (torch.Tensor): Tensor of indices specifying which
#                     tokens should be attended to by the model.
#         """
#         # Create empty lists to store outputs
#         input_ids = []
#         attention_masks = []
#         MAX_LEN = self.opt.MAX_QUESTION_LEN
#         # For every sentence...
#         for sent in data:
            
#             encoded_sent = self.tokenizer.encode_plus(
#                 text=self.text_preprocessing(sent),  # Preprocess sentence
#                 add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
#                 max_length=MAX_LEN,                  # Max length to truncate/pad
#                 pad_to_max_length=True,         # Pad sentence to max length
#                 #return_tensors='pt',           # Return PyTorch tensor
#                 truncation=True,
#                 return_attention_mask=True      # Return attention mask
#                 )
            
#             # Add the outputs to the lists
#             input_ids.append(encoded_sent.get('input_ids'))
#             attention_masks.append(encoded_sent.get('attention_mask'))

#         # Convert lists to tensors
#         input_ids = torch.tensor(input_ids)
#         attention_masks = torch.tensor(attention_masks)

#         return input_ids, attention_masks

In [17]:
# print(type(opt))

# tokenizer = BERTokenizer(opt)

# print(tokenizer)

In [18]:
with open('/home/arunava/VQA_Med/train_dataset_pickle/train_dataset_df.pkl', 'rb') as f:
    train_df = pickle.load(f) 
 

print(type(train_df))

<class 'pandas.core.frame.DataFrame'>


In [19]:
# train_df

In [20]:
tokenizer = Tokenizer.from_list(
            train_df.Question.unique().tolist(),
            max_len=12
        )

In [21]:
tokenizer

In [22]:

class VqaDataset(data.Dataset):
    '''
        Main class use to retrieve our dataset from pickle file.
    '''

    def __init__(self, input_dir, input_vqa, img_feat_vqa, tokenizer,transform=None, phase = 'train'):
        self.input_dir = input_dir
        self.vqa = np.load(input_dir+'/'+input_vqa, allow_pickle=True )
        self.img_feat_vqa = np.load(input_dir+'/'+img_feat_vqa, allow_pickle=True )
        
        self.vocab_size = None
        self.phase = phase
        self.tokenizer = tokenizer
        
   

    def __getitem__(self, idx):

        vqa = self.vqa
        img_feat_vqa = self.img_feat_vqa
        
        image_id = vqa['Image_id'].values[idx]
        image_feat = torch.Tensor(img_feat_vqa[image_id])
        image = vqa['image'].values[idx]
        qst2idc =  vqa['Question'].values[idx]
        # print('image_feat.shape',image_feat.shape)
        # print('image.shape',image.shape)
        # print('qst2idc',qst2idc)
        # question = self.tokenizer.encode(qst2idc)
        
        # question = {
        #     k: torch.tensor(v).unsqueeze(0) 
        #     for k, v in question.items()
        # }
        # print('question',question['input_ids'].shape)
        # input_ids, attention_mask = self.tokenizer.preprocessing_for_bert(qst2idc)
        # print('input_ids',input_ids.shape)
        # print('input_ids',input_ids)
        # print('attention_masks.shape',attention_mask.shape)
        # for v in input_ids:
        #     print(torch.tensor(v).unsqueeze(0))

        sample = { 'image_feature':image_feat ,
                    'question': qst2idc,
                    'image': image,
                    # 'Q': question['input_ids'],
                } 
        if (self.phase == 'train') or  (self.phase == 'valid'):
            ans2idc = vqa['labels'].values[idx]
            answer_text = vqa['Answer'].values[idx]
            sample['image_id'] = image_id
            sample['label'] = ans2idc
            sample['answer_text'] = answer_text
        else:
            sample['image_id'] = image_id
            
            
        return sample

    def __len__(self):

        return len(self.vqa)


def get_loader(input_dir, input_vqa_train, input_vqa_valid, img_feat_train, img_feat_valid, tokenizer, batch_size, num_workers,size=228):
    '''
        Load our dataset with dataloader for the train and valid data
    '''
    print('get_loader running')
    
    vqa_dataset = {
        'train': VqaDataset(
            input_dir=input_dir,
            input_vqa=input_vqa_train,
            img_feat_vqa=img_feat_train,
            tokenizer = tokenizer,
            phase = 'train'),
        'valid': VqaDataset(
            input_dir=input_dir,
            input_vqa=input_vqa_valid,
            img_feat_vqa=img_feat_valid,
            tokenizer=tokenizer,
            phase = 'valid')}
    

    data_loader = {
        phase: torch.utils.data.DataLoader(
            dataset=vqa_dataset[phase],
            batch_size=batch_size,
            shuffle=True,
            num_workers=num_workers,
            )
        for phase in ['train','valid']}

    return data_loader

In [23]:
input_dir = config.path_output_chd 
input_vqa_train =config.input_vqa_train 
input_vqa_valid =config.input_vqa_valid

img_feat_train = config.img_feat_train
img_feat_valid =config.img_feat_valid

saved_dir = config.path_output_chd


num_epochs = opt.NUM_EPOCHS
image_size = opt.IMG_INPUT_SIZE
num_workers = 0
batch_size = opt.BATCH_SIZE





In [24]:
data_loader = get_loader(
    input_dir = input_dir , 
    input_vqa_train = input_vqa_train, 
    input_vqa_valid = input_vqa_valid,
    img_feat_train = img_feat_train, 
    img_feat_valid = img_feat_valid,
    tokenizer = tokenizer,
    batch_size = batch_size, 
    num_workers = num_workers,
    size = image_size )

print('data_loader',data_loader['train'])






    

get_loader running
data_loader <torch.utils.data.dataloader.DataLoader object at 0x7f5ee04ad1e0>


In [25]:
def mixup_data_vqa(id,v, q, a, alpha=1.0, use_cuda=True):
    '''Returns mixed images, pairs of questions, targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = v.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    print('index',index)

    mixed_v = lam * v + (1 - lam) * v[index, :]
    # print('mixed_v',mixed_v.shape)
    id_a, id_b = id,id[index]
    a_a, a_b = a, a[index]
    # print('a_a',a_a)
    # print('a_b',a_b)
    q_a, q_b = q, q[index]
    # print('q_a',q_a)
    # print('q_b',q_b)
    
    return id_a,id_b,mixed_v, a_a, a_b, q_a, q_b, lam

In [26]:
for batch_idx, batch_sample in enumerate(data_loader['train']):
    # pass
    image_id = batch_sample['image_id']
    img = batch_sample['image'].to(device)
    image = batch_sample['image_feature'].to(device)
    questions = batch_sample['question']
    le = LabelEncoder()
    question_labels = le.fit_transform(questions)
    image_id_labels = le.fit_transform(image_id)
    question_labels = torch.as_tensor(question_labels).to(device)
    image_id_labels = torch.as_tensor(image_id_labels).to(device)
    # Q = batch_sample['Q'].to(device)
    labels = batch_sample['label'].to(device)
    label_answer_text = batch_sample['answer_text']#.to(device)
    # print('Q',Q.shape)
    print('image_shape',img.shape)
    print('image.shape',image.shape)
    print('questions.shape',len(questions))
    print('questions.shape',questions[0])
    print('question labels',question_labels)
    print()
    

    print('labels',labels.shape)
    print('label_answer_text',type(label_answer_text))
    id_a,id_b,mixed_v, a_a, a_b, q_a, q_b, lam = mixup_data_vqa(
                image_id_labels,img, question_labels, labels, alpha=0.2, use_cuda=True)
    
    print('mixed_v',)
    mixed_v = mixed_v.cpu().data.numpy()
    print('mixed_v',mixed_v.shape)
    print('a_a',a_a.shape)
    print('a_b',a_b.shape)
    # print('q_a',q_a)
    # print('q_b',q_b)
    # for index, x in np.ndenumerate(mixed_v):
    #     print(index, x)
    for index in range(len(mixed_v)):
        print('mixed_v_0',mixed_v[0])
        print('mixed_v_1',mixed_v[1])
        print('mixde_v_2',mixed_v[2])
        
    
    for index in range(len(mixed_v)):
        print('mixed_v',mixed_v[0])
        print('mixed_v',mixed_v[1])
        print('mixed_v',mixed_v[2])

    print('lam',lam)
    q_a_l = le.inverse_transform(q_a.cpu().data.numpy())
    q_b_l = le.inverse_transform(q_b.cpu().data.numpy())
    id_a_l = le.inverse_transform(id_a.cpu().data.numpy())
    id_b_l = le.inverse_transform(id_b.cpu().data.numpy())
    print('id_a_l',id_a_l)
    print('id_b_l',id_b_l)
    print('a_a',a_a)

    


    
    
    

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 10.92 GiB total capacity; 36.75 MiB already allocated; 5.38 MiB free; 38.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF